In [4]:
#c1vasa01
#t2ylsa01
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
'''
1. Muokkaa Pandas data framea siten, että valitset sieltä mielenkiintoisimmat sarakkeet, 
   jotka ovat:
   Country,Land Area, Armed Forces size,Co2-Emissions,GDP,Life expectancy,Population, Total tax rate, Unemployment rate
   Vinkki: Kysy chatGPT:ltä neuvoa, tehtävän eri vaiheissa. Sieltä saat hyviä toimivia esimerkkikoodeja käyttöösi.
'''
# Luetaan CSV-tiedosto DataFrameen
df = pd.read_csv('world-data-2023.csv', quotechar='"', skipinitialspace=True)

# Valitaan relevantit sarakkeet
selected_columns = [
    'Country', 
    'Land Area(Km2)', 
    'Armed Forces size', 
    'Co2-Emissions', 
    'GDP', 
    'Life expectancy', 
    'Population', 
    'Total tax rate', 
    'Unemployment rate'
]
df = df[selected_columns]

# Muunnetaan sarakkeiden nimet koodiystävällisempään.
df.rename(
    columns={
        'Land Area(Km2)': 'Land_Area', 
        'Armed Forces size': 'Armed_Forces_size', 
        'Co2-Emissions': 'Co2_Emissions'
    }, 
    inplace=True
)
'''
2. Taulukossa on NaN lukuja, jotka ovat ongelmiallisia maksimi arvoja haettaessa. Muuta kaikki Nan => 0.
   Taulukossa on myös $ ja % merkkejä, jotka kannattaa muuttaa tyhjiksi merkeiksi ennen laskuoperaatioita.
   Taulukon numerot on esitetty muodossa 3,003,000 eli pilkut pitää muuttaa tyhjiksi merkeiksi.
'''
df.fillna(0, inplace=True)  # Korvataan NaN arvot nollalla

# Poistetaan $, % merkit ja korvataan pilkut numeroissa
for col in df.select_dtypes(include=['object']).columns:
    df[col] = df[col].str.replace(',', '', regex=True)
    df[col] = df[col].str.replace('$', '', regex=True)
    df[col] = df[col].str.replace('%', '', regex=True)

# Muunnetaan objektityypit sopiviin numerotyyppeihin
for col in df.columns[1:]:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Tehtävät:

# a) Maa, jolla on suurimmat asevoimat
largest_armed_forces_country = df.loc[df['Armed_Forces_size'].idxmax()]['Country']

# b) Maa, jolla on eniten sotilaita verrattuna maapinta-alaan
df['Soldiers_per_km'] = df['Armed_Forces_size'] / df['Land_Area']
most_soldiers_per_km_country = df.loc[df['Soldiers_per_km'].idxmax()]['Country']

# c) Maa, jolla on eniten sotilaita verrattuna väestöön
df['Soldiers_per_capita'] = df['Armed_Forces_size'] / df['Population']
most_soldiers_per_capita_country = df.loc[df['Soldiers_per_capita'].idxmax()]['Country']

# d) Top 5 CO2-päästöt
top_co2_emitters = df.nlargest(5, 'Co2_Emissions')['Country'].tolist()

features2normalize = ['GDP', 'Life expectancy', 'Total tax rate', 'Unemployment rate']
df_normalized = df.copy()
for feature in features2normalize:
    max_value = df[feature].max()
    min_value = df[feature].min()
    df_normalized[feature] = (df[feature] - min_value) / (max_value - min_value)

# Painoarvot
weights = {
    'GDP': 0.50,
    'Life expectancy': 0.25,
    'Total tax rate': 0.15,
    'Unemployment rate': 0.10
}

# KOrjataan mahdolliset painoarvojen epätarkkuudet ja varmistetaan niiden summautuvan tasalukuun 1.
rounded_sum = round(sum(weights.values()), 10)
if rounded_sum != 1:
    raise ValueError("The sum of weights must be 1")


# Pistemäärät
df_normalized['Score'] = (
    df_normalized['GDP'] * weights['GDP'] +
    df_normalized['Life expectancy'] * weights['Life expectancy'] +
    df_normalized['Total tax rate'] * weights['Total tax rate'] +
    (1 - df_normalized['Unemployment rate']) * weights['Unemployment rate']
)

# Järjestetään haetut lopputulokset painoarvojen mukaisesti.
df_best_W = df_normalized.nlargest(5, 'Score')[['Country', 'Score']]


# Tulostetaan tulokset
print(f"a) Maat joilla on suurimmat asejoukot: {largest_armed_forces_country}")
print(f"b) Maa joilla on eniten sotilaita maan kokoon verrattuna: {most_soldiers_per_km_country}")
print(f"c) Maa jolla on eniten sotilaita väestöön verrattuna on: {most_soldiers_per_capita_country}")
print(f"d) Top 5 CO2 päästäjää: {top_co2_emitters}")
print(f"e) Top 5 valtiot joihin kannattaisi mennä töihin valmistumisen jälkeen:")
print(df_best_W)

a) Maat joilla on suurimmat asejoukot: India
b) Maa joilla on eniten sotilaita maan kokoon verrattuna: Singapore
c) Maa jolla on eniten sotilaita väestöön verrattuna on: North Korea
d) Top 5 CO2 päästäjää: ['China', 'United States', 'India', 'Russia', 'Japan']
e) Top 5 valtiot joihin kannattaisi mennä töihin valmistumisen jälkeen:
           Country     Score
36           China  0.811231
186  United States  0.798064
85           Japan  0.484668
64         Germany  0.444981
38         Comoros  0.422543
